In [4]:
import numpy as np
from itertools import combinations, product, permutations
import warnings
import time
from idQ import is_parallel, is_strictly_less_than, is_less_equal_than, generate_binary_vectors, generate_permutations, get_D_l, height_of_Q, distances2U, preserve_partial_order
from idQ import generate_DAG, generate_hasse_diagram, check_for_identity, Any_two_columns_contain_I, topo_order, incomplete_global_identifiability
import random
from expr_function import random_generate_Q, generate_canonical_matrices, binary_matrix_to_string, sort_lexicographically, random_generate_Q, prop_check, sort_binary_matrix
import sys
import itertools
import cProfile
import random
from idQ import T_mat, get_D_l, get_D

In [11]:
import numpy as np

def calculate_R_matrix(Q_matrix):
    """
    Calculate the R-matrix for a given Q-matrix.

    Args:
    Q_matrix (numpy.ndarray): A binary matrix (Q-matrix) of dimensions J x K.

    Returns:
    numpy.ndarray: The R-matrix of dimensions J x 2^K.
    """
    J, K = Q_matrix.shape  # J items and K attributes
    num_attribute_profiles = 2 ** K  # Total number of possible attribute profiles
    
    # Generate all possible attribute profiles (binary representations of numbers 0 to 2^K - 1)
    attribute_profiles = np.array([list(np.binary_repr(i, width=K)) for i in range(num_attribute_profiles)], dtype=int)
    
    # Initialize the R-matrix
    R_matrix = np.zeros((J, num_attribute_profiles), dtype=int)

    # Compute the R-matrix
    for j in range(J):
        for a in range(num_attribute_profiles):
            # Compute the XOR product and mod 2 for each entry
            R_matrix[j, a] = np.bitwise_and(Q_matrix[j, :], attribute_profiles[a, :]).sum() % 2

            
    # Get unique columns of R_matrix to form R^mcr matrix
    R_mcr_matrix, indices = np.unique(R_matrix, axis=1, return_index=True)

    # Sort the matrix based on the indices to maintain the original order
    R_mcr_matrix = R_mcr_matrix[:,np.argsort(indices)]
    
    return R_matrix, R_mcr_matrix

# Example Q-matrix
Q_matrix_example = np.array([[1, 1, 0], [1, 0, 1], [0, 1, 1]])

# Compute R-matrix
_, R_mcr_matrix_example = calculate_R_matrix(Q_matrix_example)
R_mcr_matrix_example



array([[0, 0, 1, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0]])

In [12]:
for Q in generate_canonical_matrices(3, 3):
    R, R_mcr = calculate_R_matrix(Q)
    print(Q)
    print(R_mcr)

[[0 0 1]
 [0 1 0]
 [0 1 1]]
[[0 1 0 1]
 [0 0 1 1]
 [0 1 1 0]]
[[0 0 1]
 [0 1 0]
 [1 0 0]]
[[0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1]
 [0 0 0 0 1 1 1 1]]
[[0 0 1]
 [0 1 0]
 [1 0 1]]
[[0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1]
 [0 1 0 1 1 0 1 0]]
[[0 0 1]
 [0 1 0]
 [1 1 1]]
[[0 1 0 1 0 1 0 1]
 [0 0 1 1 0 0 1 1]
 [0 1 1 0 1 0 0 1]]
[[0 0 1]
 [0 1 1]
 [1 0 1]]
[[0 1 0 1 0 1 0 1]
 [0 1 1 0 0 1 1 0]
 [0 1 0 1 1 0 1 0]]
[[0 0 1]
 [0 1 1]
 [1 1 0]]
[[0 1 0 1 0 1 0 1]
 [0 1 1 0 0 1 1 0]
 [0 0 1 1 1 1 0 0]]
[[0 0 1]
 [0 1 1]
 [1 1 1]]
[[0 1 0 1 0 1 0 1]
 [0 1 1 0 0 1 1 0]
 [0 1 1 0 1 0 0 1]]
[[0 0 1]
 [1 1 0]
 [1 1 1]]
[[0 1 0 1]
 [0 0 1 1]
 [0 1 1 0]]
[[0 1 1]
 [1 0 1]
 [1 1 0]]
[[0 1 1 0]
 [0 1 0 1]
 [0 0 1 1]]
[[0 1 1]
 [1 0 1]
 [1 1 1]]
[[0 1 1 0 0 1 1 0]
 [0 1 0 1 1 0 1 0]
 [0 1 1 0 1 0 0 1]]


In [1]:
import numpy as np

def reduce_Q_with_marking(Q):
    """
    Given a binary matrix Q (with no guarantee on unique or nonzero rows),
    this function produces a reduced matrix Q_reduced0 by removing all-zero rows
    and duplicate rows. It also returns a mapping that records for each original
    row its corresponding index in Q_reduced0. For rows that are all zero, the mapping
    value is the string 'zero'.

    Parameters:
        Q (np.ndarray): Binary matrix of shape (J, K).

    Returns:
        Q_reduced0 (np.ndarray): The reduced matrix with only unique nonzero rows.
        mapping (dict): A dictionary mapping each original row index (0-indexed)
                        to either 'zero' or an integer index in Q_reduced0.
    """
    Q = np.array(Q)
    J, K = Q.shape
    
    mapping = {}
    nonzero_rows = []
    nonzero_indices = []
    # First, mark all-zero rows.
    for i in range(J):
        if np.all(Q[i] == 0):
            mapping[i] = 'zero'
        else:
            nonzero_rows.append(Q[i])
            nonzero_indices.append(i)
    
    # Convert nonzero rows to an array.
    Q_nonzero = np.array(nonzero_rows)
    
    # Get the unique rows among nonzero rows.
    # np.unique returns the sorted unique rows, and an array 'inverse' such that
    # unique_rows[inverse] gives the original array.
    unique_rows, unique_indices, inverse_indices = np.unique(Q_nonzero, axis=0, return_index=True, return_inverse=True)
    
    # Build mapping for the nonzero rows.
    # For each nonzero row in Q (in the order of nonzero_indices), record its unique row index.
    for idx, orig_idx in enumerate(nonzero_indices):
        mapping[orig_idx] = int(inverse_indices[idx])
    
    return unique_rows, mapping

def lift_Q_from_reduced(Q, Q_reduced0_bar, mapping):
    """
    Reconstruct a full candidate Q_bar from the modified reduced matrix Q_reduced0_bar,
    using the mapping obtained from reduce_Q_with_marking.
    
    For each original row i:
      - If mapping[i] is 'zero', then Q_bar[i] is the zero vector.
      - Otherwise, Q_bar[i] is Q_reduced0_bar[mapping[i]].

    Parameters:
        Q (np.ndarray): The original Q (of shape (J, K)).
        Q_reduced0_bar (np.ndarray): A modified version of the reduced matrix.
        mapping (dict): Mapping from original row indices to indices in Q_reduced0_bar or 'zero'.
    
    Returns:
        Q_bar (np.ndarray): The reconstructed full candidate matrix (same shape as Q).
    """
    J, K = Q.shape
    Q_bar = np.zeros((J, K), dtype=Q.dtype)
    for i in range(J):
        if mapping[i] == 'zero':
            Q_bar[i] = np.zeros(K, dtype=Q.dtype)
        else:
            Q_bar[i] = Q_reduced0_bar[mapping[i]]
    return Q_bar

# Example usage:
if __name__ == '__main__':
    Q = np.array([
        [1, 0, 0, 1],   # row 0
        [0, 1, 0, 0],   # row 1
        [1, 1, 0, 1],   # row 2, generated by row 0 and row 1
        [0, 0, 1, 0],   # row 3
        [0, 1, 1, 0],   # row 4, generated by row 1 and row 3
        [0, 0, 1, 0]    # row 5, identical to row 3
    ])
    # Generate the reduced matrix and mapping.
    Q_reduced0, mapping = reduce_Q_with_marking(Q)
    print("Original Q:")
    print(Q)
    print("\nQ_reduced0:")
    print(Q_reduced0)
    print("\nMapping:")
    print(mapping)
    
    # Suppose we modify Q_reduced0 to obtain Q_reduced0_bar.
    # For demonstration, let's flip the first bit of each row in Q_reduced0.
    Q_reduced0_bar = Q_reduced0.copy()
    Q_reduced0_bar[:, 0] = 1 - Q_reduced0_bar[:, 0]
    print("\nQ_reduced0_bar (modified):")
    print(Q_reduced0_bar)
    
    # Lift Q_reduced0_bar to get Q_bar, a candidate for the original Q.
    Q_bar = lift_Q_from_reduced(Q, Q_reduced0_bar, mapping)
    print("\nLifted Q_bar:")
    print(Q_bar)


Original Q:
[[1 0 0 1]
 [0 1 0 0]
 [1 1 0 1]
 [0 0 1 0]
 [0 1 1 0]
 [0 0 1 0]]

Q_reduced0:
[[0 0 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [1 0 0 1]
 [1 1 0 1]]

Mapping:
{0: 3, 1: 1, 2: 4, 3: 0, 4: 2, 5: 0}

Q_reduced0_bar (modified):
[[1 0 1 0]
 [1 1 0 0]
 [1 1 1 0]
 [0 0 0 1]
 [0 1 0 1]]

Lifted Q_bar:
[[0 0 0 1]
 [1 1 0 0]
 [0 1 0 1]
 [1 0 1 0]
 [1 1 1 0]
 [1 0 1 0]]
